In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [2]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_760679.tsv', sep='\t')
MPRA_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,HCT116,A549
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.018,-0.048,0.017,-0.708,NaN
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-1.043,0.191,0.099,-0.262,NaN
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-1.005,0.650,-0.185,0.627,NaN
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.779,0.762,0.281,0.676,NaN
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.455,-0.643,0.066,-0.164,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760674,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,NaN,NaN,NaN,OL15,OL15,0.716,1.309,1.089,NaN,NaN
760675,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,NaN,NaN,NaN,OL15,OL15,-0.708,-0.544,-0.427,NaN,NaN
760676,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,NaN,NaN,NaN,OL15,OL15,-0.143,-0.195,-0.308,NaN,NaN
760677,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,NaN,NaN,NaN,OL15,OL15,-0.116,-0.084,-0.385,NaN,NaN


In [3]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

In [4]:
data_split_dict = {}

data_split_dict['total'] = np.ones(len(MPRA_df), dtype=bool)
data_split_dict['train'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(3, 23)) + ['X', 'Y'])])
data_split_dict['valid'] = MPRA_df['chr'].isin(['chr1'])
data_split_dict['test']  = MPRA_df['chr'].isin(['chr2'])

for cell_type in cell_types:
    ref_mean = MPRA_df[cell_types[:3]].mean(axis=1)
    diff = (MPRA_df[cell_type] - ref_mean).abs()
    threshold = np.percentile(diff.dropna(), 99)
    data_split_dict[f'{cell_type}_specific'] = diff > threshold


for key in data_split_dict:
    print(key, data_split_dict[key].sum())

keys = list(data_split_dict.keys())
for k1 in keys:
    for k2 in keys:
        data_split_dict[f'{k1}+{k2}'] = data_split_dict[k1] & data_split_dict[k2]

total 760679
train 631073
valid 70952
test 58654
K562_specific 7607
HepG2_specific 7607
SK-N-SH_specific 7607
HCT116_specific 4482
A549_specific 3195


In [5]:
VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_Sei_VEF_zscore.tsv', sep='\t')
VEF_df.describe()

,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_H3K27me3,K562_H3K4me1,K562_H3K36me3,K562_H3K9me3,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_H3K27me3,HepG2_H3K4me1,HepG2_H3K36me3,HepG2_H3K9me3,HepG2_CTCF,SK-N-SH_DNase,SK-N-SH_H3K4me3,SK-N-SH_H3K27ac,SK-N-SH_H3K27me3,SK-N-SH_H3K4me1,SK-N-SH_H3K36me3,SK-N-SH_H3K9me3,SK-N-SH_CTCF,HCT116_DNase,HCT116_H3K4me3,HCT116_H3K27ac,HCT116_H3K27me3,HCT116_H3K4me1,HCT116_H3K36me3,HCT116_H3K9me3,HCT116_CTCF,A549_DNase,A549_H3K4me3,A549_H3K27ac,A549_H3K27me3,A549_H3K4me1,A549_H3K36me3,A549_H3K9me3,A549_CTCF
count,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,0.000,760679.000,760679.000
mean,0.000,0.000,0.000,-0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,-0.000,0.000,NaN,0.000,-0.000,-0.000,0.000,-0.000,NaN,-0.000,NaN,-0.000,-0.000,0.000,0.000,0.000,-0.000,-0.000,NaN,-0.000,0.000,-0.000,0.000,NaN,-0.000,NaN,-0.000,0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,NaN,1.000,1.000,1.000,1.000,1.000,NaN,1.000,NaN,1.000,1.000,1.000,1.000,1.000,1.000,1.000,NaN,1.000,1.000,1.000,1.000,NaN,1.000,NaN,1.000,1.000
min,-3.071,-4.178,-4.665,-5.994,-5.530,-2.963,-1.741,-2.223,-2.539,-3.762,-4.128,-4.924,-4.987,-2.823,NaN,-2.453,-3.506,-4.290,-4.320,-3.525,NaN,-3.342,NaN,-2.462,-3.177,-2.987,-4.315,-3.715,-2.924,-2.844,NaN,-2.212,-3.474,-2.917,-3.403,NaN,-4.443,NaN,-3.275,-2.586
25%,-0.706,-0.689,-0.697,-0.625,-0.686,-0.747,-0.818,-0.673,-0.734,-0.721,-0.702,-0.700,-0.680,-0.782,NaN,-0.691,-0.682,-0.656,-0.645,-0.771,NaN,-0.782,NaN,-0.672,-0.687,-0.701,-0.686,-0.592,-0.721,-0.736,NaN,-0.627,-0.694,-0.677,-0.676,NaN,-0.692,NaN,-0.725,-0.677
50%,-0.202,-0.208,-0.205,-0.039,-0.102,-0.115,-0.316,-0.253,-0.210,-0.234,-0.183,-0.130,-0.071,-0.168,NaN,-0.212,-0.233,-0.248,-0.241,-0.136,NaN,-0.044,NaN,-0.240,-0.191,-0.259,-0.232,-0.175,-0.249,-0.159,NaN,-0.259,-0.145,-0.264,-0.243,NaN,-0.215,NaN,-0.193,-0.221
75%,0.493,0.481,0.495,0.573,0.594,0.641,0.613,0.392,0.543,0.525,0.527,0.636,0.626,0.658,NaN,0.467,0.483,0.388,0.380,0.700,NaN,0.754,NaN,0.412,0.456,0.444,0.448,0.361,0.493,0.619,NaN,0.317,0.497,0.395,0.423,NaN,0.512,NaN,0.561,0.420
max,6.455,5.184,5.623,6.448,4.656,4.533,3.655,8.456,5.766,6.191,6.044,4.098,4.231,4.624,NaN,8.384,5.613,6.558,6.388,4.103,NaN,4.445,NaN,8.475,7.113,6.104,7.072,6.734,4.692,4.500,NaN,9.919,6.630,6.189,6.035,NaN,4.868,NaN,6.659,8.890


In [6]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor

In [7]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']
data_splits = ['train', 'test', 'test+K562_specific', 'test+HepG2_specific', 'test+SK-N-SH_specific', 'test+HCT116_specific', 'test+A549_specific']

# train_cell_types = ['K562', 'HepG2']
train_cell_types = ['K562', 'HepG2', 'SK-N-SH']

# train
train_data_split = 'train'

X_train = []
y_train = []
for cell_type in train_cell_types:
    condition = data_split_dict[train_data_split]
    X_columns = [f'{cell_type}_{assay}' for assay in assays]
    y_column = cell_type
    
    X_train.append(VEF_df.loc[condition, X_columns])
    y_train.append(MPRA_df.loc[condition, y_column])

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
print(X_train.shape, y_train.shape)


# model = LinearRegression()
model = MLPRegressor(hidden_layer_sizes=100)
model.fit(X_train, y_train)


pred_array = []
for cell_type in cell_types:
    X_columns = [f'{cell_type}_{assay}' for assay in assays]
    X = np.array(VEF_df.loc[:, X_columns])
    y_pred = model.predict(X)
    pred_array.append(y_pred)
pred_array = np.array(pred_array).T
print(pred_array.shape)
np.save(f'Gosai_MPRA_Sei_VEF_MLP_pred_ct2.npy', pred_array)

(1893219, 4) (1893219,)


(760679, 5)


In [8]:
from joblib import dump, load

# 保存
dump(model, "Gosai_MPRA_Sei_VEF_MLP.joblib")

['Gosai_MPRA_Sei_VEF_MLP.joblib']

In [3]:
VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_AG_VEF_scalelog1pmean1.tsv', sep='\t')
VEF_df.describe()

,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_CTCF,SK-N-SH_DNase,SK-N-SH_H3K4me3,SK-N-SH_H3K27ac,SK-N-SH_CTCF,HCT116_DNase,HCT116_H3K4me3,HCT116_H3K27ac,HCT116_CTCF,A549_DNase,A549_H3K4me3,A549_H3K27ac,A549_CTCF
count,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000
mean,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
std,1.715,0.943,0.943,0.817,1.273,0.711,1.042,0.734,1.641,0.789,1.541,0.728,1.370,0.881,0.883,0.695,1.344,0.669,1.126,0.659
min,0.028,0.109,0.112,0.049,0.045,0.146,0.112,0.105,0.002,0.050,0.060,0.117,0.018,0.109,0.075,0.042,0.004,0.207,0.141,0.148
25%,0.345,0.534,0.552,0.558,0.528,0.647,0.552,0.658,0.293,0.606,0.414,0.633,0.480,0.600,0.542,0.586,0.472,0.657,0.486,0.657
50%,0.522,0.710,0.719,0.768,0.690,0.815,0.710,0.807,0.479,0.768,0.585,0.811,0.654,0.740,0.705,0.790,0.672,0.796,0.633,0.808
75%,0.889,1.069,1.029,1.127,0.968,1.065,1.002,1.052,1.021,1.041,0.883,1.090,0.959,1.027,1.072,1.147,1.020,1.048,0.981,1.077
max,18.442,11.261,9.982,11.996,19.305,10.640,14.293,10.593,28.245,10.888,20.343,12.804,16.906,10.493,8.369,6.435,18.612,7.471,11.607,7.788


In [4]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor

In [8]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']
data_splits = ['train', 'test', 'test+K562_specific', 'test+HepG2_specific', 'test+SK-N-SH_specific', 'test+HCT116_specific', 'test+A549_specific']

# for train_cell_types in [['K562'], ['HepG2'], ['SK-N-SH'], ['K562', 'HepG2'], ['K562', 'HepG2', 'SK-N-SH']]:
for train_cell_types in [['K562', 'HepG2', 'SK-N-SH']]:
    # train
    train_data_split = 'train'

    X_train = []
    y_train = []
    for cell_type in train_cell_types:
        condition = data_split_dict[train_data_split]
        X_columns = [f'{cell_type}_{assay}' for assay in assays]
        y_column = cell_type
        
        X_train.append(VEF_df.loc[condition, X_columns])
        y_train.append(MPRA_df.loc[condition, y_column])

    X_train = np.concatenate(X_train, axis=0)
    y_train = np.concatenate(y_train, axis=0)
    print(X_train.shape, y_train.shape)


    model = LinearRegression()
    # model = MLPRegressor(hidden_layer_sizes=100)
    # model = HistGradientBoostingRegressor()
    model.fit(X_train, y_train)


    # valid
    pearson_df = pd.DataFrame()

    for cell_type in cell_types:
        for data_split in data_splits:
            condition = data_split_dict[data_split]
            X_columns = [f'{cell_type}_{assay}' for assay in assays]
            y_column = cell_type

            X_valid = np.array(VEF_df.loc[condition, X_columns])
            y_valid = np.array(MPRA_df.loc[condition, y_column])

            y_pred = model.predict(X_valid)

            r = pearson(y_valid, y_pred)[0]
            pearson_df.loc[cell_type, data_split] = r

    print(f'trained on {train_cell_types}')
    print(f'model architecture: {type(model).__name__}')
    print(pearson_df)

(1893219, 4) (1893219,)
trained on ['K562', 'HepG2', 'SK-N-SH']
model architecture: LinearRegression
         train  test  test+K562_specific  test+HepG2_specific  test+SK-N-SH_specific  test+HCT116_specific  test+A549_specific
K562     0.631 0.604               0.543                0.616                  0.626                 0.670               0.711
HepG2    0.558 0.528               0.510                0.564                  0.584                 0.617               0.615
SK-N-SH  0.587 0.551               0.424                0.463                  0.535                 0.628               0.539
HCT116   0.594 0.571               0.557                0.565                  0.663                 0.492               0.825
A549     0.467 0.430               0.384                0.291                  0.299                 0.617               0.475
